In [1]:
import numpy as np
import math
import scipy.io as sio
import scipy.sparse as sp
from scipy.sparse import csc_matrix
import csv
import matplotlib.pyplot as plt
import datetime
import time
import os
from scipy import interpolate
import scipy.spatial.distance as distance
from collections import OrderedDict
import copy
import sys
import scipy.spatial.distance as DIST
import scipy.cluster.hierarchy as HAC
import random
from sklearn import metrics
import operator
import utils
import pickle
import partition
import clustering_worker
import utils
import vector_computation

%matplotlib inline

traj=partition.loadData('trajectory')

In [75]:
def traj_timebin(traj, timebin):
    traj_timebin=[]
    for i in range(len(traj)):
        tmp=[]
        for j in range(len(traj[i])):
            if traj[i][j][0] >timebin[1]:
                break
            if traj[i][j][0] >=timebin[0] and traj[i][j][0] <=timebin[1]:
                tmp.append(traj[i][j])
        traj_timebin.append(tmp)
    return traj_timebin

        
timebin=[1201930000,1201950000]
traj_timebin=traj_timebin(traj, timebin)

In [58]:
def tf_partition(traj):
    all_endpoint=[]
    endpoint=[]
    
    for i in range(len(traj)):
        print(i)
        endpoint=[]
        startindex=0
        length=1
        cost_np=0
        end=0 # functional terminal signal
        while startindex+length<=len(traj[i])-1:
            if startindex+length==len(traj[i])-1:
                currindex=startindex+length
                break
            currindex=startindex+length
            ldh=0            
            length_increment=clustering_worker.distanceBetweenTwoPoint(traj[i][startindex+length-1][1:], traj[i][startindex+length][1:])            
            cost_np+=length_increment            
            if length_increment==0:
                while length_increment==0: # handle the pausing car noise (5 meters)
                    length+=1
                    currindex=startindex+length

                    if currindex==len(traj[i])-1:  #judge the end
                        
                        end=1
                        break
                    
                    length_increment=clustering_worker.distanceBetweenTwoPoint(traj[i][currindex-1][1:], traj[i][currindex][1:])
                
                if end==1:
                    break
                else:
                    endpoint.append([startindex, currindex-1])
                    startindex=currindex-1
                    cost_np=clustering_worker.distanceBetweenTwoPoint(traj[i][startindex][1:], traj[i][startindex+1][1:])
                    length=1
                    continue

        
            t1=[traj[i][startindex][1:],traj[i][startindex+length][1:]]
            
            for j in range(length):
                t2=[traj[i][startindex+j][1:],traj[i][startindex+j+1][1:]]
                
                ldh+=clustering_worker.trajectoryDissimilarityAngular(t1, t2)
                ldh+=clustering_worker.trajectoryDissimilarityVertical(t1, t2)
            cost_p=clustering_worker.distanceBetweenTwoPoint(traj[i][startindex][1:], traj[i][startindex+length][1:])+ldh
            
            

            if cost_p>cost_np :
                endpoint.append([startindex, currindex-1])
                startindex=currindex-1
                length=1
                cost_np=clustering_worker.distanceBetweenTwoPoint(traj[i][startindex][1:], traj[i][startindex+1][1:])
            else:
                length+=1
                
        endpoint.append([startindex, currindex])
        all_endpoint.append(endpoint)
    return all_endpoint



In [77]:
endpoint=tf_partition(traj_timebin)
print(endpoint)

0
1
2
3
4
5
6
[[[0, 2], [2, 3], [3, 9]], [[0, 1], [1, 6], [6, 8], [8, 10], [10, 13], [13, 14], [14, 16], [16, 19], [19, 21], [21, 22], [22, 27], [27, 29], [29, 30], [30, 35], [35, 36], [36, 38], [38, 39], [39, 41], [41, 44], [44, 46], [46, 51], [51, 52], [52, 58], [58, 64], [64, 66], [66, 69], [69, 70], [70, 74], [74, 84], [84, 86], [86, 88]], [[0, 1], [1, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 9], [9, 13], [13, 15], [15, 24], [24, 25], [25, 27], [27, 28], [28, 29], [29, 33], [33, 37], [37, 38], [38, 40], [40, 41], [41, 43], [43, 46], [46, 50], [50, 52], [52, 53], [53, 55], [55, 56], [56, 58], [58, 59], [59, 61]], [[0, 1], [1, 7], [7, 10], [10, 11], [11, 13]], [[0, 3], [3, 13], [13, 15], [15, 17], [17, 20], [20, 23], [23, 27], [27, 29]], [[0, 16], [16, 22], [22, 28], [28, 30], [30, 31], [31, 36], [36, 38], [38, 43], [43, 49]], [[0, 3]]]


In [81]:
size1=[]
for i in range(len(endpoint)):
    size1.append(len(endpoint[i]))

size2=max(size1)
size1=len(endpoint)
print(size1,size2)

LocalDiffDensity=np.zeros((size1, size2), dtype=float) 
LocalAnomalyFactor=np.zeros((size1, size2), dtype=float) 
Ntc=np.zeros((size1, size2), dtype=int) 

def diff1(t1, t2):  # need. to be replaced 
    return clustering_worker.trajectoryDissimilarityAngular(t1, t2)
def diff2(t1, t2):# need. to be replaced 
    return clustering_worker.trajectoryDissimilarityVertical(t1, t2)


D=3  # parameter
for i in range(len(traj)):
    for traj_i in range(len(endpoint[i])):
        sigma_diff2=0
        t1=partition.subtrajectoryfromst2ed(traj, [i, endpoint[i][traj_i]])
        for j in range(len(traj)):
            for traj_j in range(len(endpoint[j])):
                t2=partition.subtrajectoryfromst2ed(traj, [j, endpoint[j][traj_j]])  
                if diff1(t1, t2)<D :
                    Ntc[i][traj_i]+=1
                    sigma_diff2+=diff2(t1, t2)
                    
        LocalDiffDensity[i][traj_i]=Ntc[i][traj_i]/sigma_diff2
    
for i in range(len(traj)):
    for traj_i in range(len(endpoint[i])):
        tmp=0
        t1=partition.subtrajectoryfromst2ed(traj, [i, endpoint[i][traj_i]])
        for j in range(len(traj)):
            for traj_j in range(len(endpoint[j])):
                t2=partition.subtrajectoryfromst2ed(traj, [j, endpoint[j][traj_j]])  
                if diff1(t1, t2)<D:
                    tmp+=LocalDiffDensity[j][traj_j]
        LocalAnomalyFactor[i][traj_i]=tmp/(Ntc[i][traj_i]*LocalDiffDensity[i][traj_i])

7 31


In [82]:
LocalAnomalyFactor

array([[2.89672477, 2.67284966, 0.87538397, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.50947407, 0.55685249, 1.91206599, 1.50481931, 0.49891759,
        0.51160025, 1.94258993, 0.6534823 , 0.48215638, 1.7667556 ,
        1.95149364, 2.0167896 , 1.08343416, 1.39315102, 1.90339459,
        1.82500913, 1.86977487, 1.54942431, 1.90998389, 1.94539366,
        1.2230328 , 1.76510243, 0.51055939, 0.47661193, 2.44632324,
        0.63209777, 2.16803886, 0.68749417, 0.52934384, 0.72240587,
        1.94860937],
       [2.36832968, 1.63560545, 2.37568919, 2.64680975, 0.56575688,
        1.68871027, 0.51664878, 0.48090528, 1.61120445, 0.40469857,
      

In [83]:
p=0.5    # !!!!!!parameter
part_outlier=np.zeros((size1,size2),dtype=int) 

for i in range(len(traj)):  #
    for traj_i in range(len(endpoint[i])):
        if LocalAnomalyFactor[i][traj_i]>1:
            part_outlier[i][traj_i]=1

In [98]:
def g(x):  #monotone function
    return x

ts=1
tc=2
timebins=[ts, tc]

LAF=np.zeros((size1),dtype=int) 
EAF=np.zeros((size1, tc-ts),dtype=int) 

t=0.5
for i in range(len(traj)):    
    LAF[i]=sum(LocalAnomalyFactor[i])   
    EAF[i][0]=LAF[i]
    
for i in range(len(traj)): 
    for tk in range(1, tc-ts):
        EAF[tk]=EAF[tk-1]*g(tk-1)/g(tc-ts)+LAF[i]

print(EAF)



[[ 6]
 [40]
 [40]
 [ 4]
 [19]
 [11]
 [ 5]]
